<a href="https://colab.research.google.com/github/datle2403/datle2403/blob/master/pythonProject/SVM/Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
from sklearn.datasets import fetch_openml
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
f = plt.figure()
f.set_figwidth(100)
f.set_figheight(100)

def standardized(dataset):
    scaler = StandardScaler()
    scaler.fit(dataset)
    dataset = scaler.transform(dataset)
    return dataset
def create_dataset():
    df=pd.read_csv('mnist_784_csv.csv')
    df.dropna(inplace=True)
    return df
def split(df):
  y=df['class'].to_numpy()
  X=df.drop('class',axis=1).to_numpy()
  return X,y
def pca(data,num):
    pca = PCA(n_components=num) # 95% of variance equal to 330 Principal component
    pca.fit(data)
    data = pca.transform(data)
    print(np.sum(pca.explained_variance_ratio_) )
    return data

<Figure size 7200x7200 with 0 Axes>

In [28]:
df=create_dataset()
X,y=split(df)
X.shape # 70K ex, 784 feature


(70000, 784)

In [29]:
y

array([5, 0, 4, ..., 4, 5, 6])

In [30]:

X_pca=pca(X,100)

0.9146638863541573


In [31]:
X_pca.shape

(70000, 100)

In [32]:
#split test_train set
def split_train_test(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
  return X_train, X_test, y_train, y_test
def convert_to_fit(X,y):
  feature=X.shape[1]
  W=np.random.randn(feature+1, 10)*0.001 # 331 feature, with 1 feature is b
  one=np.empty((X.shape[0],1))
  one.fill(1)
  X_pc=np.hstack((X,one))
  X_pc=X_pc.T
  return X_pc,y,W

In [33]:
def svm_loss_vectorized(W, X, y, reg):
    d, C = W.shape 
    _, N = X.shape 
    loss = 0 
    dW = np.zeros_like(W)
    
    Z = W.T.dot(X)     
    
    correct_class_score = np.choose(y, Z).reshape(N,1).T
    margins = np.maximum(0, Z - correct_class_score + 1) 
    margins[y, np.arange(margins.shape[1])] = 0.
    loss = np.sum(margins, axis = (0, 1))
    loss /= N 
    loss += 0.5 * reg * np.sum(W * W)
    
    F = (margins > 0).astype(int)
    F[y, np.arange(F.shape[1])] = np.sum(-F, axis = 0)
    dW = X.dot(F.T)/N + reg*W
    return loss, dW
def multiclass_svm_GD(X, y, Winit, reg, lr=.1, \
        batch_size = 100, num_iters = 1000, print_every = 100):
    W = Winit 
    loss_history = np.zeros((num_iters))
    for it in range(num_iters):
        # randomly pick a batch of X
        idx = np.random.choice(X.shape[1], batch_size)
        X_batch = X[:, idx]
        y_batch = y[idx]

        loss_history[it], dW = \
            svm_loss_vectorized(W, X_batch, y_batch, reg)

        W -= lr*dW 
        if it % print_every == 1:
            print ('it %d/%d, loss = %f' \
                %(it, num_iters, loss_history[it]))

    return W, loss_history 
def run(X,y,Winit,iter,reg,lr):
  W, loss_history =multiclass_svm_GD(X, y, Winit, reg, lr=lr, \
        batch_size = 500, num_iters =iter, print_every = 10)
  return W
def predict(X,y,W):
  N=X.shape[1]
  Z = W.T.dot(X)
  Z=Z.T
  r=[]
  for x in Z:
    r.append(np.argmax(x,axis=0))
  count=0
  for m in range(0,N):
    if r[m]==y[m]:
      count+=1
  score=(count/N)
  return score

In [34]:
X_train, X_test, y_train, y_test=split_train_test(X_pca,y)
print(X_train.shape)
X_train,y_train,W_train=convert_to_fit(X_train,y_train)
X_test,y_test,W_test=convert_to_fit(X_test,y_test)
W1=run(X_train,y_train,W_train,5000,0.1,0.00001)
print('score for train:',predict(X_train,y_train,W1))
print('score for test:',predict(X_test,y_test,W1))




(46900, 100)
it 1/5000, loss = 2.542935
it 11/5000, loss = 1.077483
it 21/5000, loss = 0.835389
it 31/5000, loss = 0.635999
it 41/5000, loss = 0.599394
it 51/5000, loss = 0.596737
it 61/5000, loss = 0.574257
it 71/5000, loss = 0.403892
it 81/5000, loss = 0.803564
it 91/5000, loss = 0.508283
it 101/5000, loss = 0.578158
it 111/5000, loss = 0.510758
it 121/5000, loss = 0.558411
it 131/5000, loss = 0.445244
it 141/5000, loss = 0.474949
it 151/5000, loss = 0.463721
it 161/5000, loss = 0.398252
it 171/5000, loss = 0.452888
it 181/5000, loss = 0.397055
it 191/5000, loss = 0.516117
it 201/5000, loss = 0.426090
it 211/5000, loss = 0.379507
it 221/5000, loss = 0.402915
it 231/5000, loss = 0.442085
it 241/5000, loss = 0.387076
it 251/5000, loss = 0.473510
it 261/5000, loss = 0.585422
it 271/5000, loss = 0.343443
it 281/5000, loss = 0.517413
it 291/5000, loss = 0.659529
it 301/5000, loss = 0.421912
it 311/5000, loss = 0.407888
it 321/5000, loss = 0.515815
it 331/5000, loss = 0.454088
it 341/5000,